In [36]:
from scipy.special import roots_jacobi, eval_jacobi
import numpy as np
import pandas as pd
import scipy
import math

## Define 2D Structured Multi-Element Array:

In [5]:
L = 12 # Denotes total length in [0, L]
M = 4 # Denotes number of elements 
x_range = np.arange(0, L + (L/M), (L/M)) # Define the element endpoints

In [7]:
npts = 4
p_order = npts - 1
xinterior, w = roots_jacobi(p_order - 1,1,1) # returns interior GLL nodes from range -1 and 1
GLL_points = np.pad(xinterior, (1, 1), 'constant', constant_values=(-1, 1))

In [6]:
def int_points(x_range, npts, dx, M):
    
    k = 0
    p_order = npts - 1
    xinterior, w = roots_jacobi(p_order - 1,1,1) # returns interior GLL nodes from range -1 and 1
    GLL_points = np.pad(xinterior, (1, 1), 'constant', constant_values=(-1, 1))
    GLL_dist = np.array([np.abs(GLL_points[0] - value) for value in GLL_points])
    
    return_array = np.zeros([M, npts])
    
    for i in x_range[:-1]:
        array = np.array([i])
        for j in GLL_dist[1:]: 
            scaled_point = ((j / 2) * (dx)) + i # Note: 2 here is the distance from -1 to 1
            array = np.concatenate((array, np.array([scaled_point])))
            
        return_array[k, :] = array
        k = k + 1
    
    return(return_array)

In [49]:
def array_2d(M, npts):
    return(np.zeros((M * npts, npts, npts)))

In [ ]:
# Input to this function is array_2d; returns array with indices of face-sharing neighbors:

def find_neighbors(array_in, M, ):
    

## Define 2D Gaussian Function:

In [71]:
# Note: The returned 2D array will have duplicate degrees of freedom at element boundaries:

def gaussian_2d(array_2d, coords_2d, mu, sigma_array):
    
    return_array = np.copy(array_2d).reshape(len(coords_2d), len(coords_2d))
    
    for i in np.arange(len(coords_2d)):

        # Calculate the 2D Gaussian function
        return_array[:, i] = np.exp(-(((coords_2d[i] - mu)**2 / (2 * sigma_array[0]**2)) +
                                     (coords_2d[:] - mu)**2 / (2 * sigma_array[1]**2)))
    
    #return (return_array.reshape(array_2d.shape))
    return (return_array)

In [77]:
gaussian_2d(array_2d(M, npts), int_points(x_range, 4, L/M, M).reshape(16 * 4, 4), 
            (L/2), [1, 1])

ValueError: cannot reshape array of size 16 into shape (64,4)

In [72]:
gaussian_2d(array_2d(M, npts), int_points(x_range, 4, L/M, M).reshape(16, ), 
            (L/2), [1, 1])

array([[2.31952283e-16, 2.38094992e-14, 9.97131913e-12, 1.69189792e-10,
        1.69189792e-10, 1.44344880e-09, 1.07994632e-08, 1.52299797e-08,
        1.52299797e-08, 1.07994632e-08, 1.44344880e-09, 1.69189792e-10,
        1.69189792e-10, 9.97131913e-12, 2.38094992e-14, 2.31952283e-16],
       [2.38094992e-14, 2.44400375e-12, 1.02353860e-09, 1.73670384e-08,
        1.73670384e-08, 1.48167513e-07, 1.10854615e-06, 1.56333098e-06,
        1.56333098e-06, 1.10854615e-06, 1.48167513e-07, 1.73670384e-08,
        1.73670384e-08, 1.02353860e-09, 2.44400375e-12, 2.38094992e-14],
       [9.97131913e-12, 1.02353860e-09, 4.28653704e-07, 7.27324341e-06,
        7.27324341e-06, 6.20519376e-05, 4.64254511e-04, 6.54716507e-04,
        6.54716507e-04, 4.64254511e-04, 6.20519376e-05, 7.27324341e-06,
        7.27324341e-06, 4.28653704e-07, 1.02353860e-09, 9.97131913e-12],
       [1.69189792e-10, 1.73670384e-08, 7.27324341e-06, 1.23409804e-04,
        1.23409804e-04, 1.05287518e-03, 7.87730522e-03, 1.110

## Define Derivative Operator:

In [12]:
def vandermonde(npts_values, return_type):
    # For the monomial basis: 
    if return_type == 'Monomial':
        return(np.vander(npts_values, increasing = True))
    elif return_type == 'Legendre':
        return(np.polynomial.legendre.legvander(npts_values, len(npts_values) - 1))

In [13]:
def Ld_vandermonde(npts_values): 
    
    LP = np.polynomial.legendre
    v_matrix = np.polynomial.legendre.legvander(npts_values, len(npts_values)-1)
    d_v_matrix = np.zeros_like(v_matrix)

    for i in range(len(npts_values)):
        # Coefficients for the i-th Legendre polynomial (e.g., [0, 0, 1] for P_2)
        coeffs = np.zeros(len(npts_values))
        coeffs[i] = 1

        # Compute the derivative of the i-th Legendre polynomial
        deriv_coeffs = LP.legder(coeffs, m=1) # m=1 for first derivative

        # Evaluate the derivative at the points x
        d_v_matrix[:, i] = LP.legval(npts_values, deriv_coeffs)
        
    return(d_v_matrix)

In [14]:
def return_D(V1, V2):
    return(np.matmul(V2, np.linalg.inv(V1)))

In [15]:
V1 = vandermonde(GLL_points, 'Legendre')
V2 = Ld_vandermonde(GLL_points)
D = return_D(V1, V2)

## Gradient/Divergence operators:

In [ ]:
def grad(array_in, D):
    
    dy = np.matmul(D, array_in)
    dx = np.matmul(D, array_in.transpose())

In [ ]:
def div():